In [1]:
import os
os.chdir("../")
%pwd

'd:\\Bhavadeesh\\Education\\MLOps-Udemy\\projects\\data_science_project'

In [2]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/saibhavadeesh/data_science_project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'saibhavadeesh'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '168a072a928c94440c3e0ade0a3b6eba20bb5b8d'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    all_params: dict
    metrics_file_path: Path
    target_column: str
    mlflow_uri: str

In [ ]:
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            test_data_path=config.test_data_path,
            all_params=params,
            metrics_file_path=config.metrics_file_path,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/saibhavadeesh/data_science_project.mlflow",
        )
        return model_evaluation_config

In [5]:
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, y_true, y_pred):
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        # Load the test data
        test_data = pd.read_csv(self.config.test_data_path)
        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]

        # Load the model
        model = joblib.load(self.config.model_path)

        # Make predictions
        y_pred = model.predict(X_test)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)
            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Saving the metrics to a JSON file
            metrics = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path=Path(self.config.metrics_file_path), data=metrics)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry doesnot work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [6]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_data_ingestion_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-27 20:44:05,785: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-27 20:44:05,788: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-27 20:44:05,792: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-27 20:44:05,794: INFO: common: Directory created at: artifacts]
[2025-04-27 20:44:05,795: INFO: common: Directory created at: artifacts/model_evaluation]
[2025-04-27 20:44:06,540: INFO: common: JSON file saved successfully at: artifacts\model_evaluation\metrics.json]


2025/04/27 20:44:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/04/27 20:44:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run nervous-dog-63 at: https://dagshub.com/saibhavadeesh/data_science_project.mlflow/#/experiments/0/runs/e20bd0aac3f94c15a08194c7b68e2f24
🧪 View experiment at: https://dagshub.com/saibhavadeesh/data_science_project.mlflow/#/experiments/0
